<a href="https://colab.research.google.com/github/iust-deep-learning/tensorflow-2-tutorial/blob/master/part_01_tf_architecture/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part #1: TensorFlow Architecture


TensorFlow 2.0 Tutorial by IUST

*   Last Update: Jan 2020
*   Official Page: https://github.com/iust-deep-learning/tensorflow-2-tutorial




## 1. Computation Graph

---



### What is Computational Graph
The backbone of every TensorFlow program is what we call a **computational graph**. A computational graph is a way of thinking about mathematical expressions. Lets say we have the following expression:
</br>
$$
h = (a + b)* (c + d)
$$

There are three operations in this expression, two additions and one multiplication. We can show this expression using the following graph:
</br>
</br>
<p align="center">
<img src="https://raw.githubusercontent.com/kazemnejad/tensorflow-2-tutorial/master/resources/part_01_comp_graph.png" width="200" />
<p align="left">
As you can see, every node in graph represents an operation or an input and every edge, which in TensorFlow we call a tensor, represents the data flow between these nodes (so in case you wonder where the name TensorFlow comes from, this is your answer). Almost every mathematical expression can be shown like this and it is not limited to neural networks. In a TensorFlow prgoram, we are simply creating a computational graph, hence we can benefit using TensorFlow in computing any kind of mathematical expression.
</br>
<p align="left">
But what are the advantages of a computational graph and why do we bother expressing our problem in this form in the first place?
</br>
The reason lies in the fact that some of our operations have nothing to do with each other and are completely independent. For example, in this case, two additions are independent. Using this graph form allows us to find these independent operations and compute their results in parallel. 
</br>
Besides, recall the fact that nural networks use **back propagation** to optimize the loss function and this algorithm uses **chain rule** to compute the drivatives of loss function with respect to hidden variables. Therefore, TensorFlow benefits from this graph form which makes implementation of back propagation much easier.
<p align="left">


### TensorFlow behind the scenes

<!-- $\color{red}{\text{Note:}}$ Some of these concepts, such as **session**, are deprecated and replaced in the high level python API of TensorFlow2.0, but for the sake of understanding what happens behind the scenes in TensorFlow, introducing these concepts can be helpfull.  -->

There are several layers of abstraction from hardware to the high level API of TensorFlow which we are going to use. In order to have a better sense about what happens in the background, we briefly review these layers of abstraction here.


**1.   High Level API - Making Graph**

Now, consider a simple logistic regression problem. In order to represent the problem in graph form, user creates the following graph using high level API:

</br>
</br>
<p align="center">
<img src="https://raw.githubusercontent.com/kazemnejad/tensorflow-2-tutorial/master/resources/part_01_logireg_compgraph.png" width="400" />
</br>
<p align="left">
As we discussed, nodes in this graph are either a variable, constant, an input or an operation. We discuss each of these node types later in detail, but for now, variables are parameters which can be altered by the operations to which they are passed, in fact they represent the state of our problem, and inputs are just our inputs!
<p align="left">
When we call for one of these nodes outputs, TensorFlow calculates all the requiered inputs recursively. Therefore, all the other operations which this particular operation depends on, are called first by TensorFlow.
<p align="left"> 
The question is, what happens when we pass the loss node to an optimizer and call it?
</br>  
First of all, when we add an operation using the high level API, another operation is automatically added to the graph in the background, which is responsible for calculating the output gradient of that operation with respect to its inputs.  
<br>
Secondly, each Optimizer in TensorFlow is just an operation that takes as input all the gradient operations of other nodes and variables and changes each variable using chain rule. So, optimizers are nothing but another operations.
<br/>
<br/>

**2.Distributed Master**

After the graph is created by the user, it is serialized by TensorFlow and passed to the an agent called **distributed master**. After that, the client asks distributed master for the evaluation of requested nodes using a C API.  The main task of the distributed master is to prune the graph to obtain the subgraph required to evaluate the nodes requested by the client. For example, if we ask for the output of ADD node, it is the responsibility of the distributed master to recongnize that just the output of MUL operation is needed and prune other unnecessary nodes. Besides that, distibuted master finds graph pieces which are independent and can be evaluated in parallel.
<br/>
<br/>

**3.Worker Service**

After obtaining sub-graphs, the distributed master then ships the graph pieces to the distributed tasks. The worker service in each task:

* handles requests from the master,
* schedules the execution of the kernels for the operations that comprise a local subgraph, and
* mediates direct communication between tasks.
<br/>
<br/>

**4.Kernel Implementations**

The runtime contains over 200 standard operations including mathematical, array manipulation, control flow, and state management operations. Each of these operations can have kernel implementations optimized for a variety of devices. Many of the operation kernels are implemented using C++ to generate efficient parallel code for multicore CPUs, and libraries like cuDNN for GPUs.


## References


---





*   Broadcasting in Python https://www.youtube.com/watch?v=tKcLaGdvabM
*   https://colab.research.google.com/notebooks/mlcc/creating_and_manipulating_tensors.ipynb#scrollTo=ocwT0iXH-nhT
*   https://tensorflow.org
*   http://web.stanford.edu/class/cs20si/

